<center><img src="https://github.com/hse-ds/iad-applied-ds/blob/master/2021/hw/hw1/img/logo_hse.png?raw=1" width="1000"></center>

<h1><center>Прикладные задачи анализа данных</center></h1>
<h2><center>Домашнее задание 4: рекомендательные системы</center></h2>

# Введение

В этом задании Вы продолжите работать с данными из семинара [Articles Sharing and Reading from CI&T Deskdrop](https://www.kaggle.com/gspmoreira/articles-sharing-reading-from-cit-deskdrop).

# Загрузка и предобработка данных

In [1]:
import pandas as pd
import numpy as np
import math

Загрузим данные и проведем предобраотку данных как на семинаре.

In [2]:
!kaggle datasets download -d gspmoreira/articles-sharing-reading-from-cit-deskdrop
!unzip articles-sharing-reading-from-cit-deskdrop.zip -d articles

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
unzip:  cannot find or open articles-sharing-reading-from-cit-deskdrop.zip, articles-sharing-reading-from-cit-deskdrop.zip.zip or articles-sharing-reading-from-cit-deskdrop.zip.ZIP.


In [5]:
articles_df = pd.read_csv("shared_articles.csv")
articles_df = articles_df[articles_df["eventType"] == "CONTENT SHARED"]
articles_df.head(2)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en


In [6]:
interactions_df = pd.read_csv("users_interactions.csv")
interactions_df.head(2)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US


In [7]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

In [8]:
# зададим словарь определяющий силу взаимодействия
event_type_strength = {
   "VIEW": 1.0,
   "LIKE": 2.0, 
   "BOOKMARK": 2.5, 
   "FOLLOW": 3.0,
   "COMMENT CREATED": 4.0,  
}

interactions_df["eventStrength"] = interactions_df.eventType.apply(lambda x: event_type_strength[x])

Оставляем только тех пользователей, которые произамодействовали более чем с пятью статьями.

In [10]:
users_interactions_count_df = (
    interactions_df
    .groupby(["personId", "contentId"])
    .first()
    .reset_index()
    .groupby("personId").size())
print("# users:", len(users_interactions_count_df))

users_with_enough_interactions_df = \
    users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[["personId"]]
print("# users with at least 5 interactions:",len(users_with_enough_interactions_df))

# users: 1895
# users with at least 5 interactions: 1140


Оставляем только те взаимодействия, которые относятся к отфильтрованным пользователям.

In [11]:
interactions_from_selected_users_df = interactions_df.loc[np.in1d(interactions_df.personId,
            users_with_enough_interactions_df)]

In [12]:
print(f"# interactions before: {interactions_df.shape}")
print(f"# interactions after: {interactions_from_selected_users_df.shape}")

# interactions before: (72312, 9)
# interactions after: (69868, 9)


Объединяем все взаимодействия пользователя по каждой статье и сглажиываем полученный результат, взяв от него логарифм.

In [13]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = (
    interactions_from_selected_users_df
    .groupby(["personId", "contentId"]).eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(["personId", "contentId"])
)
interactions_full_df["last_timestamp"] = (
    interactions_from_selected_users_df
    .groupby(["personId", "contentId"])["timestamp"].last()
)
        
interactions_full_df = interactions_full_df.reset_index()
interactions_full_df.head(5)

,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.000000,1470395911
1,-1007001694607905623,-6623581327558800021,1.000000,1487240080
2,-1007001694607905623,-793729620925729327,1.000000,1472834892
3,-1007001694607905623,1469580151036142903,1.000000,1487240062
4,-1007001694607905623,7270966256391553686,1.584963,1485994324


Разобьём выборку на обучение и контроль по времени.

In [14]:
from sklearn.model_selection import train_test_split

split_ts = 1475519530
interactions_train_df = interactions_full_df.loc[interactions_full_df.last_timestamp < split_ts].copy()
interactions_test_df = interactions_full_df.loc[interactions_full_df.last_timestamp >= split_ts].copy()

print(f"# interactions on Train set: {len(interactions_train_df)}")
print(f"# interactions on Test set: {len(interactions_test_df)}")

interactions_train_df

# interactions on Train set: 29329
# interactions on Test set: 9777


,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.0,1470395911
2,-1007001694607905623,-793729620925729327,1.0,1472834892
6,-1032019229384696495,-1006791494035379303,1.0,1469129122
7,-1032019229384696495,-1039912738963181810,1.0,1459376415
8,-1032019229384696495,-1081723567492738167,2.0,1464054093
...,...,...,...,...
39099,997469202936578234,9112765177685685246,2.0,1472479493
39100,998688566268269815,-1255189867397298842,1.0,1474567164
39101,998688566268269815,-401664538366009049,1.0,1474567449
39103,998688566268269815,6881796783400625893,1.0,1474567675


Для удобства подсчёта качества запишем данные в формате, где строка соответствует пользователю, а столбцы будут истинными метками и предсказаниями в виде списков.

In [20]:
interactions = (
    interactions_train_df
    .groupby("personId")["contentId"].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={"contentId": "true_train"})
    .set_index("personId")
)

interactions["true_test"] = (
    interactions_test_df
    .groupby("personId")["contentId"].agg(lambda x: list(x))
)

# заполнение пропусков пустыми списками
interactions.loc[pd.isnull(interactions.true_test), "true_test"] = [
    "" for x in range(len(interactions.loc[pd.isnull(interactions.true_test), "true_test"]))]

interactions.head(1)

,true_train,true_test
personId,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72..."


# Библиотека LightFM

Для рекомендации Вы будете пользоваться библиотекой [LightFM](https://making.lyst.com/lightfm/docs/home.html), в которой реализованы популярные алгоритмы. Для оценивания качества рекомендации, как и на семинаре, будем пользоваться метрикой *precision@10*.

In [26]:
!pip install lightfm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 310 kB 5.4 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705395 sha256=b5a0f09a15110b3dcfdef5959fad24a0378abc1a86f322146d3a587054cb2cec
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built lightfm


In [27]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

## Задание 1 (2 балла)

Модели в LightFM работают с разреженными матрицами. Создайте разреженные матрицы `data_train` и `data_test` (размером количество пользователей на количество статей), такие что на пересечении строки пользователя и столбца статьи стоит сила их взаимодействия, если взаимодействие было, и стоит ноль, если взаимодействия не было.

In [70]:
# Ваш код здесь
from scipy.sparse import csr_matrix


all_users = set(interactions_full_df['personId'])
all_items = set(interactions_full_df['contentId'])


def sparse_matrix(df, all_users, all_items):
  A = pd.DataFrame(0, index = all_users, columns = all_items)
  for row in df.values:
    A.loc[row[0], row[1]] = row[2]
  return csr_matrix(A) 


data_train = sparse_matrix(interactions_train_df, all_users, all_items)
data_test = sparse_matrix(interactions_test_df, all_users, all_items)

In [71]:
len(all_users), len(all_items)

(1140, 2984)

In [72]:
data_train.shape, data_test.shape

((1140, 2984), (1140, 2984))

## Задание 2 (1 балл)

Обучите модель LightFM с `loss="warp"` и посчитайте *precision@10* на тесте.

In [77]:
# Ваш код здесь
model = LightFM(loss='warp')
model.fit(data_train, epochs=10)

In [81]:
precision = precision_at_k(model, data_test, data_train, 10)

In [83]:
precision.mean()

0.0069246436

## Задание 3 (3 балла)

При вызове метода `fit` LightFM позволяет передавать в `item_features` признаковое описание объектов. Воспользуемся этим. Будем получать признаковое описание из текста статьи в виде [TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF) (можно воспользоваться `TfidfVectorizer` из scikit-learn). Создайте матрицу `feat` размером количесвто статей на размер признакового описание и обучите LightFM с `loss="warp"` и посчитайте precision@10 на тесте.

In [88]:
# Ваш код здесь
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
feat = tfidf.fit_transform(articles_df['text'])

model.fit(data_train, item_features=feat, epochs=10)

In [90]:
precision = precision_at_k(model, data_test, data_train, 10, item_features=feat)
precision.mean()

0.0054989816

## Задание 4 (2 балла)

В задании 3 мы использовали сырой текст статей. В этом задании необходимо сначала сделать предобработку текста (привести к нижнему регистру, убрать стоп слова, привести слова к номральной форме и т.д.), после чего обучите модель и оценить качество на тестовых данных.

In [156]:
import nltk
import string
import re
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import advertools as adv
from nltk.stem.porter import PorterStemmer

Сурс препроцессинга текста: https://www.geeksforgeeks.org/text-preprocessing-in-python-set-1/

In [126]:
set(articles_df['lang'])

{'en', 'es', 'ja', 'la', 'pt'}

Так как есть объявления на разных языках, то объединим стоп слова по им всем. По некоторым языкам стоп слов в либе stopword нет, поэтому их пришлось вынести отдельно.

In [137]:
stop_latin = {'ab', 'ac', 'ad', 'adhic', 'aliqui', 'aliquis', 'an', 'ante', 'apud', 'at', 'atque', 'aut', 'autem', 'cum', 'cur', 'de', 'deinde', 'dum', 'ego', 'enim', 'ergo', 'es', 'est', 'et', 'etiam', 'etsi', 'ex', 'fio', 'haud', 'hic', 'iam', 'idem', 'igitur', 'ille', 'in', 'infra', 'inter', 'interim', 'ipse', 'is', 'ita', 'magis', 'modo', 'mox', 'nam', 'ne', 'nec', 'necque', 'neque', 'nisi', 'non', 'nos', 'o', 'ob', 'per', 'possum', 'post', 'pro', 'quae', 'quam', 'quare', 'qui', 'quia', 'quicumque', 'quidem', 'quilibet', 'quis', 'quisnam', 'quisquam', 'quisque', 'quisquis', 'quo', 'quoniam', 'sed', 'si', 'sic', 'sive', 'sub', 'sui', 'sum', 'super', 'suus', 'tam', 'tamen', 'trans', 'tu', 'tum', 'ubi', 'uel', 'uero', 'unus', 'ut'}

In [138]:
stop_japan = adv.stopwords['japanese']

In [199]:
stop_words = set(stopwords.words("portuguese")) | set(stopwords.words("english")) | set(stopwords.words("spanish")) | stop_latin | stop_japan

In [200]:
stemmer = PorterStemmer()


def preprocessing_text(input):
  input = input.lower() # перевели в нижний регистр
  result = re.sub(r'\d+', '', input) # удалили числа
  translator = str.maketrans('', '', string.punctuation) # пунктуацию
  result = result.translate(translator) # пунктуацию
  result = " ".join(result.split()) # удалили пропуски
  result = word_tokenize(result)
  result = [stemmer.stem(word) for word in result if word not in stop_words] # выкидываем стоп слова + стемминг

  return result

In [201]:
tfidf = TfidfVectorizer(tokenizer=preprocessing_text)
feat = tfidf.fit_transform(articles_df['text'])

In [202]:
model = LightFM(loss='warp')
model.fit(data_train, item_features=feat, epochs=10)

In [203]:
precision = precision_at_k(model, data_test, data_train, 10, item_features=feat)
precision.mean()

0.007230143

Улучшилось ли качество предсказания?

Да, качество немного улучшилось.

## Задание 5 (2 балла)

Подберите гиперпараметры модели LightFM (`n_components` и др.) для улучшения качества модели.

In [212]:
# Ваш код здесь
from sklearn.model_selection import GridSearchCV


no_components = [5, 10, 25, 35, 50]
learning_rate = [0.001, 0.01, 0.05, 0.1, 0.5]

In [213]:
for no_com in no_components:
  for lr in learning_rate:
    model = LightFM(loss='warp', no_components = no_com, learning_rate = lr)
    model.fit(data_train, item_features=feat, epochs=10)
    prec = precision_at_k(model, data_test, data_train, 10, item_features=feat).mean()
    print(f'n_components={no_com}, learning_rate={lr}: precision is {prec}')
    print('------------------------------------------------')

n_components=5, learning_rate=0.001: precision is 0.0025458247400820255
------------------------------------------------
n_components=5, learning_rate=0.01: precision is 0.0033604889176785946
------------------------------------------------
n_components=5, learning_rate=0.05: precision is 0.006008146330714226
------------------------------------------------
n_components=5, learning_rate=0.1: precision is 0.005091649480164051
------------------------------------------------
n_components=5, learning_rate=0.5: precision is 0.0037678207736462355
------------------------------------------------
n_components=10, learning_rate=0.001: precision is 0.0024439918342977762
------------------------------------------------
n_components=10, learning_rate=0.01: precision is 0.0047861505299806595
------------------------------------------------
n_components=10, learning_rate=0.05: precision is 0.007331975270062685
------------------------------------------------
n_components=10, learning_rate=0.1: prec

Получаем, что для точности 0.0077 нужны параметры no_components=50 и lr = 0.05

## Бонусное задание (3 балла)

Выше мы использовали достаточно простое представление текста статьи в виде TF-IDF. В этом задании Вам нужно представить текст статьи (можно вместе с заголовком) в виде эмбеддинга полученного с помощью рекуррентной сети или трансформера (можно использовать любую предобученную модель, которая Вам нравится). Обучите модель с ипользованием этих эмеддингов и сравните результаты с предыдущими.

In [ ]:
# Ваш код здесь